In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import validators

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import urllib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from tqdm import tqdm

In [33]:
nfl_data = pd.read_csv("./nfl-elo/nfl_elo.csv")
nfl_data['year'] = nfl_data['date'].apply(lambda x: int(x.split("-")[0]))
#nfl_data['month'] = nfl_data['date'].apply(lambda x: int(x.split("-")[1]))

game_data = nfl_data[(nfl_data['year'] == 2010) | \
                     (nfl_data['year'] == 2011) | \
                     (nfl_data['year'] == 2012) | \
                     (nfl_data['year'] == 2013) | \
                     (nfl_data['year'] == 2014) | \
                     (nfl_data['year'] == 2015) | \
                     (nfl_data['year'] == 2016) | \
                     (nfl_data['year'] == 2017) | \
                     (nfl_data['year'] == 2018) | \
                     (nfl_data['year'] == 2019) | \
                     (nfl_data['year'] == 2020) | \
                     (nfl_data['year'] == 2021) | \
                     (nfl_data['year'] == 2022)
                    ]
game_data = game_data[27:]

In [3]:
teams = [
    'BUF',
    'BAL',
    'HOU',
    'DEN',
    'MIA',
    'CIN',
    'IND',
    'KAN',
    'NWE',
    'CLE',
    'JAX',
    'LVR',
    'NYJ',
    'PIT',
    'TEN',
    'LAC',
    'DAL',
    'CHI',
    'ATL',
    'ARI',
    'NYG',
    'DET',
    'CAR',
    'LAR',
    'PHI',
    'GBP',
    'NOS',
    'SFF',
    'WAS',
    'MIN',
    'TBB',
    'SEA'
]

teams_map = {
    'Buffalo Bills': 'BUF',
    'Baltimore Ravens': 'BAL',
    'Houston Texans': 'HOU',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Cincinnati Bengals': 'CIN',
    'Indianapolis Colts': 'IND',
    'Kansas City Chiefs': 'KAN',
    'New England Patriots': 'NWE',
    'Cleveland Browns': 'CLE',
    'Jacksonville Jaguars': 'JAX',
    'Las Vegas Raiders': 'LVR',
    'Oakland Raiders': 'LVR',
    'New York Jets': 'NYJ',
    'Pittsburgh Steelers': 'PIT',
    'Tennessee Titans': 'TEN',
    'Los Angeles Chargers': 'LAC',
    'San Diego Chargers': 'LAC',
    'Dallas Cowboys': 'DAL',
    'Chicago Bears': 'CHI',
    'Atlanta Falcons': 'ATL',
    'Arizona Cardinals': 'ARI',
    'New York Giants': 'NYG',
    'Detroit Lions': 'DET',
    'Carolina Panthers': 'CAR',
    'Los Angeles Rams': 'LAR',
    'St Louis Rams': 'LAR',
    'Philadelphia Eagles': 'PHI',
    'Green Bay Packers': 'GBP',
    'New Orleans Saints': 'NOS',
    'San Francisco 49ers': 'SFF',
    'Washington Football Team': 'WAS',
    'Washington Redskins': 'WAS',
    'Minnesota Vikings': 'MIN',
    'Tampa Bay Buccaneers': 'TBB',
    'Seattle Seahawks': 'SEA'
}

month_map = {"Jan": 1, "Feb": 2, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [4]:
base_url = "https://www.sportsoddshistory.com/nfl-game-team/?tm="
previous_years = "&d=2010#2019"
valid = True
for t in teams:
    validators.url(base_url + t)
    if(not validators.url(base_url + t)):
        print("{} IS NOT VALID".format(t))
        valid = False
    if(not validators.url(base_url + t + previous_years)):
        print("{} IS NOT VALID".format(t))
        valid = False
        
if(valid):
    print("ALL URLS ARE VALID")

ALL URLS ARE VALID


## Put scraper together for all teams

In [108]:
def scrape_tables(team, tables):
    dates = []
    times = []
    current_teams = []
    opponents = []
    team_scores = []
    opp_scores = []
    game_results = []
    spreads = []
    spread_results = []
    ou_lines = []
    ou_results = []
    for table in tables:
        th_val = len(table.findAll('th'))
        if(th_val in [9,10]):
            #for i in range(len(table.findAll('td'))//th_val):
            for i in range(0, len(table.findAll('td')), th_val):

                table_td = table.findAll('td')

                # Get Date and time
                #date_td = table_td[i*th_val + 2].contents[0]
                date_td = table_td[i + 2].contents[0]
                date_str = date_td[-4:] + "-"
                #month_str = str(month_map[table.findAll('td')[i*th_val + 2].contents[0][:3]])
                month_str = str(date_td[i + 2].contents[0][:3]])
                month_str = month_str if(len(month_str) == 2) else '0'+month_str
                date_str += month_str
                date_str += "-" + date_td.split(" ")[1].split(",")[0]
                dates.append(date_str)
                #times.append(table_td[i*th_val + 3].contents[0])
                times.append(table_td[i + 3].contents[0])


                # Get teams
                current_teams.append(team)
                #opponents.append(teams_map[table_td[i*th_val + 5].contents[0].contents[0]])
                opponents.append(teams_map[table_td[i + 5].contents[0].contents[0]])

                # Get score and outcome
                #game_results.append(table_td[i*th_val + 6].contents[0][0])
                #team_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[0]))
                #opp_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[1].split(" ")[0]))
                game_results.append(table_td[i + 6].contents[0][0])
                team_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[0]))
                opp_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[1].split(" ")[0]))

                # Get spread and outcome
                #spread_results.append(table_td[i*th_val + 7].contents[0][0])
                spread_results.append(table_td[i + 7].contents[0][0])
                #if(table_td[i*th_val + 7].contents[0][2:] == 'PK'):
                if(table_td[i + 7].contents[0][2:] == 'PK'):
                    spreads.append(0.)
                else:
                    #spreads.append(float(table_td[i*th_val + 7].contents[0][2:]))
                    spreads.append(float(table_td[i + 7].contents[0][2:]))

                # Get over/under and outcome
                #ou_results.append(table_td[i*th_val + 8].contents[0][0])
                #ou_lines.append(table_td[i*th_val + 8].contents[0][2:])
                ou_results.append(table_td[i + 8].contents[0][0])
                ou_lines.append(table_td[i + 8].contents[0][2:])

    return dates, times, current_teams, opponents, game_results, team_scores, opp_scores, spread_results, spreads, ou_results, ou_lines

In [110]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []

for idx, t in tqdm(enumerate(teams)):
    url = base_url + teams[0] #+ previous_years
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)

    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2020-2021 {}".format(t))
    
    time.sleep(1)
    url = base_url + teams[0] + previous_years
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)
    
    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2010-2019 {}".format(t))

    time.sleep(5)
    
print(dates)

0it [00:00, ?it/s]

DONE WITH 2020-2021 BUF
DONE WITH 2010-2019 BUF


1it [00:20, 20.11s/it]

DONE WITH 2020-2021 BAL
DONE WITH 2010-2019 BAL


2it [00:29, 14.09s/it]

...

DONE WITH 2020-2021 TBB
DONE WITH 2010-2019 TBB


31it [05:20, 10.02s/it]

DONE WITH 2020-2021 SEA
DONE WITH 2010-2019 SEA


32it [05:30, 10.33s/it]

['2021-09-12', '2021-09-19', '2021-09-26', '2021-10-3', '2021-10-10', '2021-10-18', '2021-10-31', '2021-11-7', '2021-11-14', '2021-11-21', '2021-11-25', '2021-12-6', '2021-12-12', '2021-12-19', '2021-12-26', '2022-01-2', '2022-01-9', '2022-01-15', '2022-01-23', '2020-09-13', '2020-09-20', '2020-09-27', '2020-10-4', '2020-10-13', '2020-10-19', '2020-10-25', '2020-11-1', '2020-11-8', '2020-11-15', '2020-11-29', '2020-12-7', '2020-12-13', '2020-12-19', '2020-12-28', '2021-01-3', '2021-01-9', '2021-01-16', '2021-01-24', '2019-09-8', '2019-09-15', '2019-09-22', '2019-09-29', '2019-10-6', '2019-10-20', '2019-10-27', '2019-11-3', '2019-11-10', '2019-11-17', '2019-11-24', '2019-11-28', '2019-12-8', '2019-12-15', '2019-12-21', '2019-12-29', '2020-01-4', '2018-09-9', '2018-09-16', '2018-09-23', '2018-09-30', '2018-10-7', '2018-10-14', '2018-10-21', '2018-10-29', '2018-11-4', '2018-11-11', '2018-11-25', '2018-12-2', '2018-12-9', '2018-12-16', '2018-12-23', '2018-12-30', '2017-09-10', '2017-09-17'

In [84]:
# Save data
np.save("./scraped_data/dates.npy", dates)
np.save("./scraped_data/times.npy", times)
np.save("./scraped_data/current_teams.npy", current_teams)
np.save("./scraped_data/opponents.npy", opponents)
np.save("./scraped_data/game_results.npy", game_results)
np.save("./scraped_data/team_scores.npy", team_scores)
np.save("./scraped_data/opp_scores.npy", opp_scores)
np.save("./scraped_data/spread_results.npy", spread_results)
np.save("./scraped_data/spreads.npy", spreads)
np.save("./scraped_data/ou_results.npy", ou_results)
np.save("./scraped_data/ou_lines.npy", ou_lines)

## Make dataset

In [170]:
scraped_data = pd.DataFrame()
scraped_data['date'] = dates
scraped_data['time'] = times
scraped_data['current_team'] = current_teams
scraped_data['opponent'] = opponents
scraped_data['game_result'] = game_results
scraped_data['team_score'] = team_scores
scraped_data['opp_score'] = opp_scores
scraped_data['spread_result'] = spread_results
scraped_data['spread'] = spreads
scraped_data['ou_result'] = ou_results
scraped_data['ou_line'] = ou_lines
scraped_data['year'] = scraped_data['date'].apply(lambda x: int(x.split("-")[0]))
min_2010 = scraped_data[(scraped_data['current_team'] == 'MIN')]
elo_min_2010 = game_data[(game_data['year'] == 2010) & ((game_data['team1']=='MIN') | (game_data['team2']=='MIN'))]

#print(min_2010['date'])
#print(elo_min_2010['date'])
#print(set(scraped_data['date'].values) & set(game_data['date'].values))

#for d in sorted(np.unique(scraped_data['date'].values)):
#    print(d)
#print("\n\n")
#for d in sorted(np.unique(game_data['date'].values)):
#    print(d)
    
print(len(sorted(np.unique(scraped_data['date'].values))))
print(len(sorted(np.unique(game_data['date'].values))))
#for g in game_data[['date', 'team1', 'team2']].iterrows():
#    try:
#        print(dates == g[1]['date'])
#    except:
#        continue

200
663


In [164]:
no_match, match = 0, 0
for idx, row in tqdm(scraped_data.iterrows()):
    date_data = game_data[game_data['date'] == row['date']]
    matched = date_data[(date_data['team1'] == row['current_team']) | (date_data['team2'] == row['current_team'])]
    if(matched.empty):
        no_match += 1
    else:
        match += 1
        
print(len(game_data))
print(match, no_match)

6400it [00:02, 2608.81it/s]

3224
2458 3942


## Debugging code

In [99]:
url = base_url + teams[29] + previous_years
req = requests.get(url)#, headers)
soup = BeautifulSoup(req.content, 'html.parser')
tables = soup.findAll(lambda tag: tag.name=='table')

In [105]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []
for table in tables:
    th_val = len(table.findAll('th'))
    if(th_val in [9,10]):
        print(len(table.findAll('td')))
        #for i in range(len(table.findAll('td'))//th_val):
        for i in range(0, len(table.findAll('td')), th_val):
            print(i)
            table_td = table.findAll('td')
            #print(table_td[i*th_val:(i+1)*th_val])
            
            # Get Date and time
            date_td = table_td[i + 2].contents[0]
            date_str = date_td[-4:] + "-"
            if(date_str not in ['2010-', '2011-']):
                continue
            month_str = str(month_map[table.findAll('td')[i + 2].contents[0][:3]])
            month_str = month_str if(len(month_str) == 2) else '0'+month_str
            date_str += month_str
            date_str += "-" + date_td.split(" ")[1].split(",")[0]
            print(date_str)
            dates.append(date_str)
            times.append(table_td[i + 3].contents[0])

            
            # Get teams
            current_teams.append("BUF")
            opponents.append(teams_map[table_td[i + 5].contents[0].contents[0]])
            
            # Get score and outcome
            game_results.append(table_td[i + 6].contents[0][0])
            team_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[0]))
            opp_scores.append(int(table_td[i + 6].contents[0][2:].split("-")[1].split(" ")[0]))

            # Get spread and outcome
            spread_results.append(table_td[i + 7].contents[0][0])
            if(table_td[i + 7].contents[0][2:] == 'PK'):
                spreads.append(0.)
            else:
                spreads.append(float(table_td[i + 7].contents[0][2:]))
            
            # Get over/under and outcome
            ou_results.append(table_td[i + 8].contents[0][0])
            ou_lines.append(table_td[i + 8].contents[0][2:])
            
print(dates)
print(times)
print(opponents)
print(game_results)
print(team_scores)
print(opp_scores)
print(spread_results)
print(spreads)
print(ou_results)
print(len(ou_lines))

160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
18
0
9
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
18
0
9
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
9
0
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
9
0
160
0
2011-09-11
10
2011-09-18
20
2011-09-25
30
2011-10-2
40
2011-10-9
50
2011-10-16
60
2011-10-23
70
2011-10-30
80
2011-11-14
90
2011-11-20
100
2011-11-27
110
2011-12-4
120
2011-12-11
130
2011-12-18
140
2011-12-24
150
160
0
2010-09-9
10
2010-09-19
20
2010-09-26
30
2010-10-11
40
2010-10-17
50
2010-10-24
60
2010-10-31
70
2010-11-7
80
2010-11-14
90
2010-11-21
100
2010-11-28
110
2010-12-5
120
2010-12-13
130
2010-12-20
140
2010-12-28
150
2011-01-2
['2011-09-11', '2011-09-18', '2011-09-25', '2011-10-2', '2011-10-9', '2011